# 计算每天的情感倾向,作为边的权重
# 每个人可能有很多条评论,结果相加

In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm
from snownlp import SnowNLP

file_names = os.listdir(os.getcwd())  # 路径
comment_file = (re.findall('user_L[0-9]+_post_comment.csv', ''.join(file_names)))
print(len(comment_file))
data=pd.DataFrame()
for file in comment_file:
    data =data.append(pd.read_csv(file, encoding='utf_8_sig'))

data['content'].replace(to_replace=' ',value='',inplace=True)
data.dropna(subset=['content'],inplace=True)
data['sentiment']=None
data['score']=None

11


# get the key of baidu api

In [12]:
import requests
import time
access_token_list=[]
api_key='2VyIDSSm72ikpdNvHEFuSZYE'
client_secret='EuqOAwzQbC4oncTKDn9mcIMTRTavYeSi'
# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = f'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={client_secret}'
response = requests.get(host)
if response:
    print(response.json())
access_token=eval(response.text)['access_token']
access_token_list.append(access_token)

api_key='PGALriW1axDoujwcN0N2LCiD'
client_secret='x9d7s0Iat2DQsFbmOmkz8c4ezkot4VHm'
host = f'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={client_secret}'
response = requests.get(host)
if response:
    print(response.json())
access_token_1=eval(response.text)['access_token']
access_token_list.append(access_token_1)

{'refresh_token': '25.c88163c0f5c12f06bcb9ddcf2fee7294.315360000.1917924559.282335-18606470', 'expires_in': 2592000, 'session_key': '9mzdWTg0XlYKZLZPI7CaeTGtvi60zSAcoIOYsVbKX6A+ctQ2UFA+LaFO2cd8gxcCyRqP/lVCcI5tR3Dj/1xbuM8LBQzoMA==', 'access_token': '24.bb5e19336b7dcd8040fd38b31127aa80.2592000.1605156559.282335-18606470', 'scope': 'brain_nlp_address public nlp_simnet nlp_wordemb nlp_comtag nlp_dnnlm_cn brain_nlp_lexer brain_all_scope brain_nlp_comment_tag brain_nlp_dnnlm_cn brain_nlp_word_emb_vec brain_nlp_word_emb_sim brain_nlp_sentiment_classify brain_nlp_simnet brain_nlp_depparser brain_nlp_wordembedding brain_nlp_dnnlm_cn_legacy brain_nlp_simnet_legacy brain_nlp_comment_tag_legacy brain_nlp_lexer_custom brain_nlp_keyword brain_nlp_topic brain_nlp_ecnet brain_nlp_emotion brain_nlp_comment_tag_custom brain_nlp_news_summary brain_nlp_sentiment_classify_custom wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test权限 vis-classify_flow

In [3]:
# 去除自己的回复自己的
# data[data['author_user_name']==data['reply1user']]
data['content']=data['content'].apply(lambda x: x.replace(' ',''))
data.drop(data[data['author_user_name']==data['reply2user']].index,inplace=True)
data.drop(data[data['content'].str.len()==0].index,inplace=True)
data.reset_index(drop=True,inplace=True)

In [4]:
import time
# host='https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag'
# host=f'https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token={access_token}'
# battery_post['post_tag']=None
data['sentiment_json']=None
data['sentiment_json'].fillna('{}',inplace=True)
data['sentiment_json']=data['sentiment_json'].apply(lambda x:eval(x))

In [58]:
for index,row in tqdm(data.iterrows()):
    try:
        if not row['sentiment_json'].get('log_id'):
            content = {'text': row['content']}
            host=f'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?access_token={access_token_list[index%2]}'
            result = requests.post(url=host,json=content)
            data.at[index,'sentiment_json']=result.text
            time.sleep(0.1)
    except BaseException:
        pass
    # break

18433it [00:06, 3025.75it/s] 


Wall time: 6.1 s


In [6]:
temporary_pd=data

In [59]:
data['sentiment_json']=data['sentiment_json'].astype('str')
data['sentiment_json']=data['sentiment_json'].apply(lambda x:eval(x))
# 查漏补缺
for index,row in tqdm(data.iterrows()):
    try:
        if row['sentiment_json'].get('error_code'):
            content = {'text': row['content']}
            host=f'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?access_token={access_token_list[index%2]}'
            result = requests.post(url=host,json=content)
            data.at[index,'sentiment_json']=result.text
            time.sleep(0.3)
    except BaseException as e :
        # print(e)
        pass
    # break

18433it [00:09, 1904.57it/s]


In [60]:
# extract the sentiment
try:
    data['sentiment_json']=data['sentiment_json'].apply(lambda x:eval(str(x)))
except BaseException as e:
    print(e)
for index,row in tqdm(data.iterrows()):
    try:
        if row['sentiment_json'].get('items'):
            data.at[index,'sentiment']=row['sentiment_json']['items'][0]['sentiment']-1
    except BaseException as e :
        pass

18433it [00:01, 10401.70it/s]


In [65]:
data.to_csv('comment.csv',index=False,encoding='utf_8_sig')

In [63]:
print(data.dtypes)
print(data[data['sentiment']=='None'])
tmp=(data[data['sentiment']=='None'])

author_user_name    object
post_id              int64
reply2user          object
content             object
date                object
sentiment           object
score               object
sentiment_json      object
dtype: object
      author_user_name     post_id reply2user  \
993        卡卡西123456go  6086825431       果断留空   
2634               NaN  5743818844  951540015   
2778          敏爷ningdu  4503006921    万达王家马德里   
5832                禮盒  4192874376     暖男欧巴丶c   
6539           星期五荔枝先生  5883131480      紫水关银屏   
7578          Cva11cgh  6386095531   海的西边2018   
7602            向往西藏33  5661089725        茹李儒   
8758     gy15603742703  6625773209   海的西边2018   
11266          全球焦点TWO  4443446524     fhj778   
11951         迷你168168  4443446524     fhj778   
12170          全球焦点TWO  4443446524     fhj778   
13041      卡卡西123456go  6086825431       果断留空   
14146           浪漫爱情主义  5202558376     fhj778   
15498           有时会很迷茫  6515512547       铁南南南   
16793             se色染  6538256871 

In [64]:
data.drop(tmp.index,inplace=True)
data['sentiment']=data['sentiment'].astype('int64')
print(data.dtypes)

author_user_name    object
post_id              int64
reply2user          object
content             object
date                object
sentiment            int64
score               object
sentiment_json      object
dtype: object


# 计算周围节点 对该节点的评价求和
# 计算每个节点 对该节点的评价

In [66]:
node=pd.read_csv(f"node.csv", encoding='utf_8_sig')
edge=pd.read_csv(f"edge.csv", encoding='utf_8_sig')
node['sentiment_sum']=0
node['sentiment_category']=0 # 取绝对值然后分类,为了方便可视化
edge['sentiment_classify']=0 # 求和之后 进行分类 -1 0
for name,group in tqdm(data.groupby('reply2user')):
    # print(name,group['sentiment'].sum())
    index= node['author_user_name']==name
    sentiment_sum=group['sentiment'].sum()
    if sentiment_sum>0:
        node.at[index,'sentiment_category']=1
    elif sentiment_sum<0:
        node.at[index,'sentiment_category']=-1
    else:
        node.at[index,'sentiment_category']=0
    node.at[index,'sentiment_sum']=abs(sentiment_sum)

    for n,g in group.groupby('author_user_name'):
        edge.loc[edge[(edge['author_user_name']==str(n))&(edge['reply2user']==str(name))].index,'sentiment_classify']=g['sentiment'].sum()
    # break
edge.loc[edge['sentiment_classify']>0,'sentiment_classify']=1
edge.loc[edge['sentiment_classify']==0,'sentiment_classify']=0
edge.loc[edge['sentiment_classify']<0,'sentiment_classify']=-1
print(node['sentiment_sum'].describe())
print(edge['sentiment_classify'].describe())

100%|██████████| 338/338 [00:51<00:00,  6.55it/s]


count    16185.000000
mean         0.374977
std         14.884514
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       1374.000000
Name: sentiment_sum, dtype: float64
count    18276.000000
mean        -0.101280
std          0.879047
min         -1.000000
25%         -1.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: sentiment_classify, dtype: float64


# 将观点求总

In [67]:
from ast import literal_eval
post_file = (re.findall('user_L[0-9]+_post_filter.csv', ''.join(file_names)))
print(len(post_file))
post=pd.DataFrame()
for file in post_file:
    post =post.append(pd.read_csv(file, encoding='utf_8_sig'))

node['prop_adj_list']=None
for name,group in tqdm(post.groupby('author_user_name')):
    tmp=[]
    for index,row in group.iterrows():
        tmp.extend(literal_eval(row['prop_adj_list']))
    tmp=list(set(tmp))
    node.at[node[node['author_user_name']==str(name)].index[0],'prop_adj_list']=tmp
    # print(name,group['sentiment'].sum())


11


100%|██████████| 423/423 [00:00<00:00, 524.01it/s]


In [69]:
# save
node.to_csv('node.csv',index=False,encoding='utf_8_sig')
edge.to_csv('edge.csv',index=False,encoding='utf_8_sig')


